In [1]:
import pandas as pd
import json
from ast import literal_eval

In [2]:
data = pd.read_csv("raw/AnimeList.csv")

In [3]:
data.columns

Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')

In [4]:
data.iloc[0]

anime_id                                                      11013
title                                                 Inu x Boku SS
title_english                             Inu X Boku Secret Service
title_japanese                                               妖狐×僕SS
title_synonyms                                      Youko x Boku SS
image_url         https://myanimelist.cdn-dena.com/images/anime/...
type                                                             TV
source                                                        Manga
episodes                                                         12
status                                              Finished Airing
airing                                                        False
aired_string                           Jan 13, 2012 to Mar 30, 2012
aired                    {'from': '2012-01-13', 'to': '2012-03-30'}
duration                                            24 min. per ep.
rating                                    PG-13 

In [5]:
project = data[['anime_id', 'title', 'episodes', 'aired', 'type', 'source', 'aired_string', 'duration', 'producer', 'studio', 'genre']]

In [6]:
project['from'] = project.aired.apply(lambda x: literal_eval(x)['from'])

In [7]:
project['to'] = project.aired.apply(lambda x: literal_eval(x)['to'])


In [8]:
project.drop('aired', axis='columns', inplace=True)

In [9]:
filtered = project[~ project['from'].apply(lambda x: x is None)]

In [10]:
filtered['year'] = filtered['from'].apply(lambda x: int(x[:4]))

In [11]:
filtered.drop('from', axis='columns', inplace=True)
filtered.drop('to', axis='columns', inplace=True)

In [12]:
grouped = filtered.groupby('year').apply(lambda x: json.loads(x.to_json(orient='records')))

In [13]:
grouped

year
1917    [{'anime_id': 10757, 'title': 'Kachikachi Yama...
1918    [{'anime_id': 10759, 'title': 'Kintarou', 'epi...
1926    [{'anime_id': 28041, 'title': 'Baguda-jou no T...
1927    [{'anime_id': 35492, 'title': 'Tako no Hone', ...
1928    [{'anime_id': 7498, 'title': 'Doubutsu Olympic...
                              ...                        
2014    [{'anime_id': 23903, 'title': 'Tuzki: Love Ass...
2015    [{'anime_id': 27417, 'title': 'Diabolik Lovers...
2016    [{'anime_id': 32370, 'title': 'D.Gray-man Hall...
2017    [{'anime_id': 34152, 'title': 'Super Danganron...
2018    [{'anime_id': 33185, 'title': 'Peace Maker Kur...
Length: 88, dtype: object

In [14]:
grouped.to_json("history.json")